## Testing Julia for GAIA

using the julia language for GAIA analaysis

In [1]:
using PyCall
using DataFrames
using Clustering
using Statistics

import PyPlot; const plt = PyPlot


@pyimport astropy.io.votable as votable
# @pyimport matplotlib.pyplot as plt

## directory
rootdir = "/home/stephane/Science/cluster/GAIA"
wdir    = "/home/stephane/Science/cluster/GAIA/products"

cd(wdir)

In [2]:
## Types and Constants

DEG2RAD =  π / 180.

mutable struct Df
    ndata::Int32
    data::Array{Float64}
    err::Array{Float64}
end

function copy(s::Df)
    c = Df(s.ndata, zeros(length(s.data[:,1]),s.ndata), zeros(length(s.err[:,1]),s.ndata))
    c.data[:,:] = s.data[:,:]
    c.err[:,:]  = s.err[:,:]
    
    return(c)
end
    

In [3]:
## function to create the df and filter the data
#########
function read_votable(voname::String)
    
    vot = votable.parse(voname)
    data = vot[:get_first_table]() 
    
    println("## Votable $voname read")
    
    return(data["array"]["data"])
end


-

filter_data (generic function with 5 methods)

In [4]:
######
function add_cartesian(s::Df, centering = true)::Df
    
    dfresult = copy(s)
    off = zeros(2)
    
    if centering
        off[1] = mean(s.data[1,:])
        off[2] = mean(s.data[2,:])
    end
    
    lgal = DEG2RAD .* (s.data[1,:] .- off[1])
    bgal = DEG2RAD .* (s.data[2,:] .- off[2])
    
    dfresult.data[1,:] = s.data[3,:] .* cos.(bgal) .* cos.(lgal)
    dfresult.data[2,:] = s.data[3,:] .* cos.(bgal) .* sin.(lgal)
    dfresult.data[3,:] = s.data[3,:] .* sin.(bgal)
    
    println("## Cartesian transformation done ...")
    
    return(dfresult)    
end

####################
function  normalization_PerBlock(s::Df, block , weightblock, norm , density = false)::Df
   
    dfresult    = copy(s)
    totalWeight = sum(weightblock)
    
    for axis in block, weight in weightblock
        normK = normalizationVector(norm, density, dfresult.data[:,axis])
        normK[1] = normK[1] / totalWeight
        dfresult.data[:,axis]    =   weight .* (s.data[:,axis] .- normK[1] ) ./ normK[2]
    end
    
    println("## Normalization $norm done...")
    return(dfresult)
end
 
#############
function normalizationVector(norm, density, arr)
        
    vecNorm = [0.0, 1.0]
        
    if norm == "identity"
        vecNorm = [0.0, 1.0]
            
        elseif norm == "normal"
            stdArr = std(arr)
            meanArr = mean(arr)
            vecNorm  = [meanArr , stdArr]
            
        elseif norm == "minmax"
            minarr  = min(arr)
            maxarr  = max(arr)
            vecNorm = [minarr, maxarr-minarr]
    end
            
    if density
        vecNorm[1] = vecNorm[1] * length(arr[:,0])
    end
        
    return(vecNorm)
end

normalizationVector (generic function with 1 method)

In [5]:
## Main program
voname = "NGC 2516-1.0deg.vot"

data       = read_votable(voname)
df         = filter_data(data)
dfcart     = add_cartesian(df)

blck       = [[1,2,3],[4,5], [6,7,8]]
wghtblck   = [3.,2.,1.5]
norm = "normal"
dfcartnorm = normalization_PerBlock(dfcart, blck, wghtblck , norm) 



## Votable NGC 2516-1.0deg.vot  read
## Filtering done ...
## Stars selected: 13557
## Cartesian transformation done ...
## Normalization normal done...


In [6]:
## Clustering ..
println("## Starting DBSCAN ...")

eps = 2.0

res = dbscan(dfcartnorm.data , eps , leafsize=20, min_neighbors=5, min_cluster_size=10)

a = assignments(res)
println(a)

#println(source.data[:,1])
x = dfcartnorm.data[1,:]
y = dfcartnorm.data[2,:]

plt.scatter(x, y , s = 0.5 )
plt.show()

## Starting DBSCAN ...


MethodError: MethodError: no method matching assignments(::Array{DbscanCluster,1})
Closest candidates are:
  assignments(!Matched::ClusteringResult) at /home/stephane/.julia/packages/Clustering/jd204/src/utils.jl:11